# PLOTTING

In [ ]:
import shadie
import toytree
import numpy as np
import pyslim
import toyplot
import itertools
import glob

In [2]:
import numpy as np
import scipy.stats

#just a random code I found for calculating confidence intervals
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [3]:
neut_chrom_small = shadie.chromosome.explicit({
    (0, 900_000): shadie.NONCDS,})

chrom_small = shadie.chromosome.explicit({
    (0, 100_000): shadie.NONCDS,
    (100_001, 120_000): shadie.EXON,
    (120_001, 130_000): shadie.INTRON,
    (130_001, 150_000): shadie.EXON,
    (150_001, 750_000): shadie.NONCDS,
    (750_001, 770_000): shadie.EXON,
    (770_001, 780_000): shadie.INTRON,
    (780_001, 800_000): shadie.EXON,
    (800_001, 900_000): shadie.NONCDS,
})
chrom_small.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfdf88dd7610d4e6db0cd00bc62ad64ac"> name: noncds
interval: (0, 100000)
ElementType: g3
coding: False name: exon
interval: (100001, 120000)
ElementType: g1
coding: True name: intron
interval: (120001, 130000)
ElementType: g2
coding: True name: exon
interval: (130001, 150000)
ElementType: g1
coding: True name: noncds
interval: (150001, 750000)
ElementType: g3
coding: False name: exon
interval: (750001, 770000)
ElementType: g1
coding: True name: intron
interval: (770001, 780000)
ElementType: g2
coding: True name: exon
interval: (780001, 800000)
ElementType: g1
coding: True name: noncds
interval: (800001, 900000)
ElementType: g3
coding: False 0 100000 200000 300000 400000 500000 600000 700000 800000 900000

In [4]:
TREEDIR = "/home/deren/Desktop/standard-params/"

MB_TREEFILES = sorted(glob.glob(TREEDIR +
    f"bryo-mono/bryo_mono_run1[0-9]_from_*.trees"))

DB_TREEFILES = sorted(glob.glob(TREEDIR +
    f"bryo-dio/bryo_dio_run1[0-9]_from_*.trees"))

Ho_TREEFILES = sorted(glob.glob(TREEDIR +
    f"pter-homospore/pter_homospore_run1[0-9]_from_*.trees"))

He_TREEFILES = sorted(glob.glob(TREEDIR +
    f"pter-hetero/pter_hetero_run2[0-9]_from_*.trees"))

MA_TREEFILES = sorted(glob.glob(TREEDIR +
    f"angio-mono/angio_mono_run1[0-9]_from_*.trees"))

TREEFILES = {
    "MB": MB_TREEFILES,
    "DB": DB_TREEFILES,
    "Ho": Ho_TREEFILES,
    "He": He_TREEFILES,
    "MA": MA_TREEFILES,
}

In [5]:
MB_TREEFILES

['/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run10_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run11_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run13_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run14_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run15_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run16_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run17_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run18_from_smallchrom_2000spo.trees',
 '/home/deren/Desktop/standard-params/bryo-mono/bryo_mono_run19_from_smallchrom_2000spo.trees']

## TWO POPS

In [ ]:
# dict to store all stats
fst_stats = {}
div_stats = {}
theta_stats = {}
taj_stats = {}

taj_done = set()
theta_done = set()

# iterate over plant types
for plant, tree_sets in TREEFILES.items():
    
    fst_stats[plant] = []
    div_stats[plant] = []
    theta_stats[plant] = []
    taj_stats[plant] = []
    
    for pair in itertools.combinations(tree_sets, 2):

        # print name of pair
        fname0 = pair[0].split("/")[-1].split("_from_smallchrom")[0]
        fname1 = pair[1].split("/")[-1].split("_from_smallchrom")[0]

        # get tree sequence pair merged
        post = shadie.postsim.src.one_sim.TwoSims(
            trees_files=[pair[0], pair[1]],
            ancestral_ne=200,
            recomb=1e-8,
            mut=1e-7,
            chromosome=chrom_small,
            recapitate=True,
        )

        # get windowed stats and store to plant key
        div = post.get_windowed_stats("divergence", window_size=25_000, sample=20, reps=100).mean(0)
        fst = post.get_windowed_stats("Fst", window_size=25_000, sample=20, reps=100).mean(0)
        theta0 = post.get_windowed_stats("theta0", window_size=25_000, sample=20, reps=100).mean(0)
        theta1 = post.get_windowed_stats("theta1", window_size=25_000, sample=20, reps=100).mean(0)
        taj0 = post.get_windowed_stats("tajimas0", window_size=25_000, sample=20, reps=100).mean(0)
        taj1 = post.get_windowed_stats("tajimas1", window_size=25_000, sample=20, reps=100).mean(0)

        # store 1pop results for either pop if not yet stored
        if fname0 not in taj_done:
            taj_stats[plant].append(taj0)
            taj_done.add(fname0)
        if fname1 not in taj_done:
            taj_stats[plant].append(taj1) 
            taj_done.add(fname1)
            
        if fname0 not in theta_done:
            theta_stats[plant].append(theta0)
            theta_done.add(fname0)
        if fname1 not in theta_done:
            theta_stats[plant].append(theta1) 
            theta_done.add(fname1)
        
        # record 2pop stats
        fst_stats[plant].append(fst)
        div_stats[plant].append(div)
        
        print(f"{fname0}, {fname1}")#", {div:.5f}, {fst:.5f}, {theta0:.5f}, {theta1:.5f}, {taj0:.5f}, {taj1:.5f}")


bryo_mono_run10, bryo_mono_run11
bryo_mono_run10, bryo_mono_run13
bryo_mono_run10, bryo_mono_run14
bryo_mono_run10, bryo_mono_run15
bryo_mono_run10, bryo_mono_run16
bryo_mono_run10, bryo_mono_run17
bryo_mono_run10, bryo_mono_run18
bryo_mono_run10, bryo_mono_run19
bryo_mono_run11, bryo_mono_run13
bryo_mono_run11, bryo_mono_run14
bryo_mono_run11, bryo_mono_run15
bryo_mono_run11, bryo_mono_run16
bryo_mono_run11, bryo_mono_run17
bryo_mono_run11, bryo_mono_run18
bryo_mono_run11, bryo_mono_run19
bryo_mono_run13, bryo_mono_run14
bryo_mono_run13, bryo_mono_run15
bryo_mono_run13, bryo_mono_run16
bryo_mono_run13, bryo_mono_run17
bryo_mono_run13, bryo_mono_run18
bryo_mono_run13, bryo_mono_run19
bryo_mono_run14, bryo_mono_run15
bryo_mono_run14, bryo_mono_run16
bryo_mono_run14, bryo_mono_run17
bryo_mono_run14, bryo_mono_run18
bryo_mono_run14, bryo_mono_run19
bryo_mono_run15, bryo_mono_run16
bryo_mono_run15, bryo_mono_run17
bryo_mono_run15, bryo_mono_run18
bryo_mono_run15, bryo_mono_run19
bryo_mono_

In [ ]:
fst_stats

In [17]:
np.concatenate(stats["MB"]).mean(axis=0)

array([0.00033723, 0.00026452, 0.00026893, 0.00024631, 0.00022413,
       0.00023792, 0.00024711, 0.00027205, 0.00019359, 0.00020137,
       0.00019288, 0.00020355, 0.00023419, 0.0002033 , 0.00019655,
       0.0002453 , 0.00020629, 0.00029254, 0.00024614, 0.0001913 ,
       0.0002542 , 0.00026769, 0.00028844, 0.00033759, 0.00025494,
       0.00021481, 0.00029993, 0.00029942, 0.00029262, 0.0004689 ,
       0.00021989, 0.00036306, 0.0003992 , 0.00026959, 0.00029774])

In [79]:
np.vstack(mstats["MB"])

array([[4.94043896e-04, 2.56063604e-04, 4.23091197e-04, ...,
        4.57039214e-04, 4.16655093e-04, 3.71446810e-04],
       [1.13983207e-04, 5.51832720e-05, 1.42916508e-04, ...,
        4.19221756e-04, 3.38605179e-04, 4.88171680e-04],
       [3.78038469e-04, 4.06543993e-04, 3.10430211e-04, ...,
        4.49360612e-05, 7.77193581e-05, 1.25047083e-04],
       ...,
       [1.65199816e-04, 1.11085988e-04, 2.38408068e-04, ...,
        4.80880021e-04, 2.89488567e-04, 2.07335881e-04],
       [4.79732800e-04, 3.25207972e-04, 3.54199606e-04, ...,
        5.72541031e-04, 3.40413511e-04, 3.28007969e-04],
       [3.97813447e-04, 3.48794057e-04, 2.10233100e-04, ...,
        6.57921491e-04, 3.87702347e-04, 3.75996804e-04]])

In [ ]:
db_vals = np.vstack(mstats["DB"])#.mean(axis=0)
mb_vals = np.vstack(mstats["MB"])#.mean(axis=0)
ma_vals = np.vstack(mstats["MA"])
he_vals = np.vstack(mstats["He"])#.mean(axis=0)
ho_vals = np.vstack(mstats["Ho"])#.mean(axis=0)

In [85]:
canvas = toyplot.Canvas(500, 600)
ax0 = canvas.cartesian(bounds=("15%", "90%", "85%", "90%"))
ax1 = canvas.cartesian(bounds=("15%", "90%", "60%", "80%"))
ax2 = canvas.cartesian(bounds=("15%", "90%", "35%", "55%"))

# draw chromosome
chrom_small.draw(axes=ax0);
ax0.x.ticks.locator = toyplot.locator.Extended(only_inside=True)

# draw divergence stats
for dat in [mb_vals, db_vals, he_vals, ho_vals, ma_vals]:
    ax1.plot(dat.mean(axis=0))
    ax1.fill(
        np.arange(dat[0].size),
        dat.mean(axis=0) - (dat.std(axis=0)),
        dat.mean(axis=0) + (dat.std(axis=0)),
        opacity=0.2,
        #color='red',
    )

stat_names = ("Divergence", "Diversity")
axes = (ax1, ax2)
for stat_name, ax in zip(stat_names, axes):
    ax.x.show = False
    ax.x.ticks.labels.show = False
    ax.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    ax.y.label.text = stat_name
    ax.y.label.offset = 50
    ax.y.spine.style = {"stroke-width": 1.5}

    ax.y.ticks.labels.angle = -90
    ax.y.ticks.show = True
    ax.y.ticks.locator = toyplot.locator.Extended(only_inside=True, count=5)  

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="600.0px" viewBox="0 0 500.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="teca88d10f9d24bac853d62f96d799cf2"> name: noncds
interval: (0, 100000)
ElementType: g3
coding: False name: exon
interval: (100001, 120000)
ElementType: g1
coding: True name: intron
interval: (120001, 130000)
ElementType: g2
coding: True name: exon
interval: (130001, 150000)
ElementType: g1
coding: True name: noncds
interval: (150001, 750000)
ElementType: g3
coding: False name: exon
interval: (750001, 770000)
ElementType: g1
coding: True name: intron
interval: (770001, 780000)
ElementType: g2
coding: True name: exon
interval: (780001, 800000)
ElementType: g1
coding: True name: noncds
interval: (800001, 900000)
ElementType: g3
coding: False 0 300000 600000 900000 <path d="M 75.0 440.4645645640758 L 86.02941176470588 449.4834425718214 L 97.05882352941177 448.9372926287214 L 108.08823529411765 451.74255083848277 L 119.11764705882352 454.4942728190038 L 130.1470588235294 452.7831236045657 L 141.1764705882353 451.6438538852026 L 152.20588235294116 448.5492119201655 L 163.23529411764704 458.28248031907003 L 174.26470588235296 457.31781978192043 L 185.2941176470588 458.3714616863178 L 196.32352941176472 457.04752882829905 L 207.3529411764706 453.24685857649484 L 218.38235294117646 457.078350687436 L 229.41176470588235 457.91563319129773 L 240.44117647058823 451.86758038011254 L 251.4705882352941 456.70808635354206 L 262.5 446.00754090966325 L 273.5294117647059 451.76446451028875 L 284.55882352941177 458.56651045142195 L 295.5882352941176 450.76426837967546 L 306.61764705882354 449.0901958516578 L 317.6470588235294 446.5170396420068 L 328.6764705882353 440.4188745081072 L 339.7058823529412 450.67236228601166 L 350.73529411764713 455.65109759726664 L 361.76470588235287 445.09153200712086 L 372.7941176470588 445.15411043177545 L 383.8235294117647 445.99842836000533 L 394.8529411764706 424.13018529419696 L 405.88235294117646 455.0205895657895 L 416.9117647058824 437.259158217866 L 427.94117647058823 432.77646052699663 L 438.9705882352941 448.8543416252179 L 450.0 445.3631060881391" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <polygon points="75.0,458.2616909408939 86.02941176470588,466.70589772738384 97.05882352941177,463.34786890386385 108.08823529411765,468.9189149921539 119.11764705882352,474.3971018855349 130.1470588235294,467.2087673126201 141.1764705882353,464.08163412485806 152.20588235294116,464.3591432127059 163.23529411764704,471.29586490652457 174.26470588235296,469.5790916133439 185.2941176470588,473.2669365801687 196.32352941176472,470.9086396006565 207.3529411764706,467.073378591921 218.38235294117646,464.52824454908887 229.41176470588235,476.2842204798163 240.44117647058823,469.9325775800666 251.4705882352941,466.92109715268606 262.5,468.66754630633176 273.5294117647059,462.7211685918776 284.55882352941177,469.45192674776195 295.5882352941176,469.0760423421614 306.61764705882354,464.2848792605215 317.6470588235294,460.03858675238615 328.6764705882353,455.86138718291704 339.7058823529412,462.632307802891 350.73529411764713,469.6773293508836 361.76470588235287,462.1583820583556 372.7941176470588,462.74516469799033 383.8235294117647,463.5469384743285 394.8529411764706,448.23409128029414 405.88235294117646,467.36803495456013 416.9117647058824,461.666079375229 427.94117647058823,456.37897165853593 438.9705882352941,461.9944467617043 450.0,459.8365402671236 450.0,430.8896719091546 438.9705882352941,435.71423648873156 427.94117647058823,409.17394939545727 416.9117647058824,412.85223706050283 405.88235

In [43]:
canvas = toyplot.Canvas(500, 600)
ax0 = canvas.cartesian(bounds=("15%", "90%", "85%", "90%"))
ax1 = canvas.cartesian(bounds=("15%", "90%", "60%", "80%"))
ax2 = canvas.cartesian(bounds=("15%", "90%", "35%", "55%"))

# draw chromosome
chrom_small.draw(axes=ax0);
ax0.x.ticks.locator = toyplot.locator.Extended(only_inside=True)

# draw divergence stats
for dat in [mb_vals, he_vals, ho_vals]:
    ax1.plot(dat.mean(axis=0))
    ax1.fill(
        np.arange(dat[0].size),
        dat.mean(axis=0) - (dat.std(axis=0)),
        dat.mean(axis=0) + (dat.std(axis=0)),
        opacity=0.2,
        #color='red',
    )

stat_names = ("Divergence", "Diversity")
axes = (ax1, ax2)
for stat_name, ax in zip(stat_names, axes):
    ax.x.show = False
    ax.x.ticks.labels.show = False
    ax.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    ax.y.label.text = stat_name
    ax.y.label.offset = 50
    ax.y.spine.style = {"stroke-width": 1.5}

    ax.y.ticks.labels.angle = -90
    ax.y.ticks.show = True
    ax.y.ticks.locator = toyplot.locator.Extended(only_inside=True, count=5)  

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="600.0px" viewBox="0 0 500.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2819156b43454248b1489e1d77a53333"> name: noncds
interval: (0, 100000)
ElementType: g3
coding: False name: exon
interval: (100001, 120000)
ElementType: g1
coding: True name: intron
interval: (120001, 130000)
ElementType: g2
coding: True name: exon
interval: (130001, 150000)
ElementType: g1
coding: True name: noncds
interval: (150001, 750000)
ElementType: g3
coding: False name: exon
interval: (750001, 770000)
ElementType: g1
coding: True name: intron
interval: (770001, 780000)
ElementType: g2
coding: True name: exon
interval: (780001, 800000)
ElementType: g1
coding: True name: noncds
interval: (800001, 900000)
ElementType: g3
coding: False 0 300000 600000 900000 <path d="M 75.0 440.5178419205512 L 86.02941176470588 449.5243632360026 L 97.05882352941177 448.97896156855165 L 108.08823529411765 451.7803763165792 L 119.11764705882352 454.52832818492385 L 130.1470588235294 452.81952340227593 L 141.1764705882353 451.68181458720176 L 152.20588235294116 448.5914125662563 L 163.23529411764704 458.31134549233155 L 174.26470588235296 457.3480066288329 L 185.2941176470588 458.40020494692294 L 196.32352941176472 457.0780859986756 L 207.3529411764706 453.2826230146459 L 218.38235294117646 457.10886562903113 L 229.41176470588235 457.9450009788616 L 240.44117647058823 451.90523455623855 L 251.4705882352941 456.7391085913259 L 262.5 446.05322387878385 L 273.5294117647059 451.8022599646393 L 284.55882352941177 458.5949864772812 L 295.5882352941176 450.80343419470825 L 306.61764705882354 449.1316553000108 L 317.6470588235294 446.5620245510077 L 328.6764705882353 440.47221446416154 L 339.7058823529412 450.7116540208452 L 350.73529411764713 455.68356800685274 L 361.76470588235287 445.1384699926792 L 372.7941176470588 445.2009626791368 L 383.8235294117647 446.04412381415716 L 394.8529411764706 424.2058422524685 L 405.88235294117646 455.0539238293637 L 416.9117647058824 437.3168272758218 L 427.94117647058823 432.84027129304184 L 438.9705882352941 448.89612421555137 L 450.0 445.40967199223104" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> <polygon points="75.0,458.31540831473086 86.02941176470588,466.74965168335297 97.05882352941177,463.3957520951896 108.08823529411765,468.94954620896635 119.11764705882352,474.4055572771866 130.1470588235294,467.22778867574334 141.1764705882353,464.1239242727905 152.20588235294116,464.4278645152449 163.23529411764704,471.33661392207927 174.26470588235296,469.6308246338317 185.2941176470588,473.2974138001564 196.32352941176472,470.96335823334067 207.3529411764706,467.1297641136967 218.38235294117646,464.61168695961237 229.41176470588235,476.3096678105347 240.44117647058823,469.9826934254067 251.4705882352941,466.9807330303922 262.5,468.7706818318638 273.5294117647059,462.7961454630055 284.55882352941177,469.52381811737797 295.5882352941176,469.22056183840016 306.61764705882354,464.4560382627768 317.6470588235294,460.12852448031504 328.6764705882353,455.97817074829686 339.7058823529412,462.71014425909004 350.73529411764713,469.73454712415213 361.76470588235287,462.2205956993484 372.7941176470588,462.83316585919823 383.8235294117647,463.66040482485755 394.8529411764706,448.3691503811822 405.88235294117646,467.4384857824711 416.9117647058824,461.72073745896824 427.94117647058823,456.4665189073244 438.9705882352941,462.07139142514364 450.0,459.93825937505306 450.0,430.8810846094089 438.9705882352941,435.72085700595915 427.94117647058823,409.214023678759 416.9117647058824,412.91291709267534 405.8

In [129]:
data = []
means = []
for post in posts:
    c, a, m = post.draw_stats(stat="divergence", window_size=20_000, sample=20, reps=100, color="steelblue");
    data.append(post.rep_values)
    means.append(post.means)

NameError: name 'rep_values' is not defined

In [20]:
data = []
means = []
for post in posts:
    post.draw_stats(stat="divergence", window_size=25_000, sample=20, reps=100, color = "steelblue");
    data.append(post.rep_values)
    means.append(post.means)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t15761cad49e5472cb53c826f4a34cf84"> <polygon points="80.0,250.0 90.88235294117646,250.0 101.76470588235294,250.0 112.64705882352942,250.0 123.52941176470588,250.0 134.41176470588235,250.0 145.29411764705884,250.0 156.1764705882353,250.0 167.05882352941177,250.0 177.94117647058823,250.0 188.8235294117647,250.0 199.7058823529412,250.0 210.58823529411765,250.0 221.4705882352941,250.0 232.35294117647058,250.0 243.23529411764707,250.0 254.1176470588235,250.0 265.0,250.0 275.88235294117646,250.0 286.7647058823529,250.0 297.6470588235294,250.0 308.52941176470586,250.0 319.4117647058824,250.0 330.29411764705884,250.0 341.1764705882353,250.0 352.0588235294118,250.0 362.9411764705882,250.0 373.8235294117647,250.0 384.70588235294116,250.0 395.5882352941176,250.0 406.47058823529414,250.0 417.3529411764706,250.0 428.235294117647,250.0 439.11764705882354,250.0 450.0,250.0 450.0,120.70809403203367 439.11764705882354,104.94310277173341 428.235294117647,90.58143973174487 417.3529411764706,81.19394998751477 406.47058823529414,168.63339635429665 395.5882352941176,60.0 384.70588235294116,140.45161060179083 373.8235294117647,240.1519637570007 362.9411764705882,214.66093532622253 352.0588235294118,216.45025505654053 341.1764705882353,176.8480005707559 330.29411764705884,112.96115292690766 319.4117647058824,159.4224663789106 308.52941176470586,163.97709841971968 297.6470588235294,176.04822887311383 286.7647058823529,185.72004423358186 275.88235294117646,176.74633467698777 265.0,214.59315806371066 254.1176470588235,173.09314022758895 243.23529411764707,209.13031070524025 232.35294117647058,219.4798986908287 221.4705882352941,199.56016123854022 210.58823529411765,214.70837940998112 199.7058823529412,233.25223843327507 188.8235294117647,224.93596832304783 177.94117647058823,208.49320443762707 167.05882352941177,241.3855099347198 156.1764705882353,178.48821032354726 145.29411764705884,189.95612314058428 134.41176470588235,235.89555167124462 123.52941176470588,246.72635822066852 112.64705882352942,157.2197053472693 101.76470588235294,102.82845218135768 90.88235294117646,160.66956800913215 80.0,77.92708593443443" style="fill:rgb(27.5%,51%,70.6%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.0000 0.0003 0.0005 divergence in 25kb windows

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t919ad2e82b6e4fc182a8d0a182250795"> <polygon points="80.0,250.0 90.88235294117646,250.0 101.76470588235294,250.0 112.64705882352942,250.0 123.52941176470588,250.0 134.41176470588235,250.0 145.29411764705884,250.0 156.1764705882353,250.0 167.05882352941177,250.0 177.94117647058823,250.0 188.8235294117647,250.0 199.7058823529412,250.0 210.58823529411765,250.0 221.4705882352941,250.0 232.35294117647058,250.0 243.23529411764707,250.0 254.1176470588235,250.0 265.0,250.0 275.88235294117646,250.0 286.7647058823529,250.0 297.6470588235294,250.0 308.52941176470586,250.0 319.4117647058824,250.0 330.29411764705884,250.0 341.1764705882353,250.0 352.0588235294118,250.0 362.9411764705882,250.0 373.8235294117647,250.0 384.70588235294116,250.0 395.5882352941176,250.0 406.47058823529414,250.0 417.3529411764706,250.0 428.235294117647,250.0 439.11764705882354,250.0 450.0,250.0 450.0,180.95558546433384 439.11764705882354,228.82769092728716 428.235294117647,180.16219760499706 417.3529411764706,140.95834627463168 406.47058823529414,210.6322255582013 395.5882352941176,210.19291161956033 384.70588235294116,217.04489767746838 373.8235294117647,191.02702143079 362.9411764705882,206.46857852779777 352.0588235294118,205.3211167477655 341.1764705882353,163.67808951927398 330.29411764705884,142.13203575249335 319.4117647058824,138.53228422542026 308.52941176470586,193.55143734686135 297.6470588235294,141.64026641819373 286.7647058823529,229.9948234806916 275.88235294117646,180.54249922352196 265.0,72.16965179280129 254.1176470588235,191.16471684439387 243.23529411764707,204.59329813300207 232.35294117647058,193.00065569244578 221.4705882352941,205.74731683749175 210.58823529411765,171.92014356213554 199.7058823529412,79.21834558442913 188.8235294117647,129.37226075853266 177.94117647058823,96.35814611588489 167.05882352941177,113.97004520826863 156.1764705882353,168.72036442695935 145.29411764705884,178.46395417054896 134.41176470588235,143.83683611139864 123.52941176470588,182.96200434827625 112.64705882352942,156.57694033198754 101.76470588235294,111.43907236773997 90.88235294117646,178.3918280015184 80.0,60.0" style="fill:rgb(27.5%,51%,70.6%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.0000 0.0003 0.0005 divergence in 25kb windows

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta50a2d7e0eb94d3e97598c1710749b8b"> <polygon points="80.0,250.0 90.88235294117646,250.0 101.76470588235294,250.0 112.64705882352942,250.0 123.52941176470588,250.0 134.41176470588235,250.0 145.29411764705884,250.0 156.1764705882353,250.0 167.05882352941177,250.0 177.94117647058823,250.0 188.8235294117647,250.0 199.7058823529412,250.0 210.58823529411765,250.0 221.4705882352941,250.0 232.35294117647058,250.0 243.23529411764707,250.0 254.1176470588235,250.0 265.0,250.0 275.88235294117646,250.0 286.7647058823529,250.0 297.6470588235294,250.0 308.52941176470586,250.0 319.4117647058824,250.0 330.29411764705884,250.0 341.1764705882353,250.0 352.0588235294118,250.0 362.9411764705882,250.0 373.8235294117647,250.0 384.70588235294116,250.0 395.5882352941176,250.0 406.47058823529414,250.0 417.3529411764706,250.0 428.235294117647,250.0 439.11764705882354,250.0 450.0,250.0 450.0,103.22592005634802 439.11764705882354,109.52691201502608 428.235294117647,176.0832306157187 417.3529411764706,90.40089217585276 406.47058823529414,227.6509948934672 395.5882352941176,204.33174854727946 384.70588235294116,128.4744966836884 373.8235294117647,107.1849504020661 362.9411764705882,157.69325585490367 352.0588235294118,133.42607266537533 341.1764705882353,173.73011680460172 330.29411764705884,86.83218876562779 319.4117647058824,154.19146563362094 308.52941176470586,135.1212067852321 297.6470588235294,226.24581792569114 286.7647058823529,67.89575629512217 275.88235294117646,199.27980278217981 265.0,178.64823619181783 254.1176470588235,246.77701473264074 243.23529411764707,120.7794799553912 232.35294117647058,180.54410987849968 221.4705882352941,221.48382931267236 210.58823529411765,131.4409813934377 199.7058823529412,137.08399366085578 188.8235294117647,225.46516405470447 177.94117647058823,156.0984915184597 167.05882352941177,157.23601573046895 156.1764705882353,245.13764160356868 145.29411764705884,114.3223572225154 134.41176470588235,115.71638199213456 123.52941176470588,183.06450666197082 112.64705882352942,155.4293596290426 101.76470588235294,113.21828960497717 90.88235294117646,156.52227504842392 80.0,60.0" style="fill:rgb(27.5%,51%,70.6%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.0000 0.0001 0.0002 0.0003 divergence in 25kb windows

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3b869da3ceef4b9989b43d22ba61634d"> <polygon points="80.0,250.0 90.88235294117646,250.0 101.76470588235294,250.0 112.64705882352942,250.0 123.52941176470588,250.0 134.41176470588235,250.0 145.29411764705884,250.0 156.1764705882353,250.0 167.05882352941177,250.0 177.94117647058823,250.0 188.8235294117647,250.0 199.7058823529412,250.0 210.58823529411765,250.0 221.4705882352941,250.0 232.35294117647058,250.0 243.23529411764707,250.0 254.1176470588235,250.0 265.0,250.0 275.88235294117646,250.0 286.7647058823529,250.0 297.6470588235294,250.0 308.52941176470586,250.0 319.4117647058824,250.0 330.29411764705884,250.0 341.1764705882353,250.0 352.0588235294118,250.0 362.9411764705882,250.0 373.8235294117647,250.0 384.70588235294116,250.0 395.5882352941176,250.0 406.47058823529414,250.0 417.3529411764706,250.0 428.235294117647,250.0 439.11764705882354,250.0 450.0,250.0 450.0,140.6429017021656 439.11764705882354,181.8905623384725 428.235294117647,140.38552713661883 417.3529411764706,60.0 406.47058823529414,161.91016843418598 395.5882352941176,202.44666250779787 384.70588235294116,155.85509312895465 373.8235294117647,175.16834506728455 362.9411764705882,145.2214597629441 352.0588235294118,176.3265306122449 341.1764705882353,157.013278673915 330.29411764705884,153.447286338116 319.4117647058824,120.77087603600393 308.52941176470586,170.91827822832192 297.6470588235294,103.07976116210679 286.7647058823529,198.52508689065147 275.88235294117646,223.17208804919343 265.0,205.95169770965157 254.1176470588235,229.48453791997147 243.23529411764707,209.79538365564565 232.35294117647058,193.66544871223599 221.4705882352941,191.5353355315924 210.58823529411765,110.82470368059887 199.7058823529412,128.54629712146868 188.8235294117647,137.65600213884682 177.94117647058823,119.49077622315295 167.05882352941177,69.85812316192857 156.1764705882353,143.85669726405848 145.29411764705884,137.05320381427663 134.41176470588235,152.96978878887796 123.52941176470588,161.86275733000627 112.64705882352942,134.78424382853566 101.76470588235294,189.59148026022632 90.88235294117646,163.32234203725153 80.0,184.9722841101506" style="fill:rgb(27.5%,51%,70.6%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.0000 0.0005 0.0010 divergence in 25kb windows

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tab16d0cc883f4ae380572f0e4bea015c"> <polygon points="80.0,250.0 90.88235294117646,250.0 101.76470588235294,250.0 112.64705882352942,250.0 123.52941176470588,250.0 134.41176470588235,250.0 145.29411764705884,250.0 156.1764705882353,250.0 167.05882352941177,250.0 177.94117647058823,250.0 188.8235294117647,250.0 199.7058823529412,250.0 210.58823529411765,250.0 221.4705882352941,250.0 232.35294117647058,250.0 243.23529411764707,250.0 254.1176470588235,250.0 265.0,250.0 275.88235294117646,250.0 286.7647058823529,250.0 297.6470588235294,250.0 308.52941176470586,250.0 319.4117647058824,250.0 330.29411764705884,250.0 341.1764705882353,250.0 352.0588235294118,250.0 362.9411764705882,250.0 373.8235294117647,250.0 384.70588235294116,250.0 395.5882352941176,250.0 406.47058823529414,250.0 417.3529411764706,250.0 428.235294117647,250.0 439.11764705882354,250.0 450.0,250.0 450.0,205.01921965680978 439.11764705882354,232.78672461874731 428.235294117647,168.8216968570208 417.3529411764706,188.52674421526018 406.47058823529414,177.5810868527498 395.5882352941176,157.67555209406325 384.70588235294116,146.30505238298622 373.8235294117647,213.33944677536869 362.9411764705882,198.7706956762051 352.0588235294118,217.29191266989926 341.1764705882353,194.226314599402 330.29411764705884,140.0135668165716 319.4117647058824,185.58148883250004 308.52941176470586,139.29276687686854 297.6470588235294,160.52533728613423 286.7647058823529,223.5356631409693 275.88235294117646,120.54528553124108 265.0,136.85827701429534 254.1176470588235,101.16197271562439 243.23529411764707,95.9397532849282 232.35294117647058,155.87593899957287 221.4705882352941,120.53430645931178 210.58823529411765,166.27264276562065 199.7058823529412,64.57302213400997 188.8235294117647,127.6597241413963 177.94117647058823,90.66502524935277 167.05882352941177,60.0 156.1764705882353,135.56465593045746 145.29411764705884,157.27935080270316 134.41176470588235,175.52847775293327 123.52941176470588,211.79760319573887 112.64705882352942,179.24704168027532 101.76470588235294,191.63429892219168 90.88235294117646,235.56968067733587 80.0,241.84207220561265" style="fill:rgb(27.5%,51%,70.6%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.0000 0.0003 0.0005 0.0008 divergence in 25kb windows

In [24]:
cis = []
for dat in data:
    ci = mean_confidence_interval(dat)
    cis.append(ci)

In [25]:
import toyplot
#plot in toyplot
canvas = toyplot.Canvas(width=500, height=300)
axes = canvas.cartesian()
colors = ["gold", "mediumseagreen", "purple", "red", "steelblue"]

for i in range(0,4):
    fill = axes.fill(np.arange(35),  means[i]-cis[i][1],  means[i]+cis[i][2], style = {"fill":colors[i], "fill-opacity":0.1,})
    line = axes.plot(means[i], style={"stroke":colors[i], "stroke-width":1, "stroke-opacity":0.5})

# style axes
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
axes.y.ticks.labels.angle = -90
axes.y.ticks.show = True
axes.y.ticks.locator = toyplot.locator.Extended(only_inside=True, count=5)        
axes.label.offset = 20
axes.label.text = f"divergence in {int(25000 / 1000)}kb windows"

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taedbb188b8ee4936a2f49912bc714dec"> <polygon points="50.0,182.55172249683113 61.76470588235294,205.9897081329725 73.52941176470588,189.61398436827372 85.29411764705883,205.00905768182676 97.05882352941177,230.37870131290302 108.8235294117647,227.36130557703507 120.58823529411765,214.2697171550074 132.35294117647058,211.0383638222216 144.11764705882354,228.8350297430674 155.88235294117646,219.4940694483976 167.6470588235294,224.16369285278884 179.41176470588238,226.56006655010646 191.1764705882353,221.26797059923433 202.9411764705882,216.91338466824635 214.70588235294116,222.66128816842343 226.47058823529412,219.63682575502656 238.23529411764704,209.4970167975945 250.0,221.22485429969143 261.7647058823529,210.48293694742634 273.52941176470586,213.05675291086428 285.2941176470588,210.3572239245448 297.05882352941177,206.9322619029887 308.8235294117647,205.60995727384682 320.5882352941177,192.37082356223155 332.3529411764706,210.58683946015088 344.1176470588236,221.77013505474454 355.8823529411764,221.26951381260983 367.6470588235294,228.450381241066 379.4117647058823,200.26081371007808 391.1764705882353,177.61425237187623 402.94117647058823,208.35499673211623 414.7058823529412,183.50052352205643 426.4705882352941,186.1356385529436 438.2352941176471,190.21538647226697 450.0,194.70121433114932 450.0,151.51085005511436 438.2352941176471,147.02502219623204 426.4705882352941,142.9452742769086 414.7058823529412,140.3101592460215 402.94117647058823,165.1646324560813 391.1764705882353,134.4238880958413 379.4117647058823,157.07044943404318 367.6470588235294,185.26001696503107 355.8823529411764,178.07914953657487 344.1176470588236,178.57977077870964 332.3529411764706,167.39647518411593 320.5882352941177,149.1804592861966 308.8235294117647,162.4195929978119 297.05882352941177,163.74189762695374 285.2941176470588,167.16685964850984 273.52941176470586,169.86638863482935 261.7647058823529,167.29257267139138 250.0,178.03449002365647 238.23529411764704,166.30665252155956 226.47058823529412,176.4464614789916 214.70588235294116,179.4709238923885 202.9411764705882,173.72302039221142 191.1764705882353,178.0776063231994 179.41176470588238,183.36970227407156 167.6470588235294,180.9733285767539 155.88235294117646,176.30370517236264 144.11764705882354,185.64466546703247 132.35294117647058,167.84799954618666 120.58823529411765,171.07935287897246 108.8235294117647,184.1709413010001 97.05882352941177,187.1883370368681 85.29411764705883,161.81869340579178 73.52941176470588,146.42362009223876 61.76470588235294,162.79934385693755 50.0,139.36135822079618" style="fill:rgb(100%,84.3%,0%);fill-opacity:0.1;opacity:1.0;stroke:none"> <path d="M 50.0 161.11148581284579 L 61.76470588235294 184.51554458211004 L 73.52941176470588 168.1549419005159 L 85.29411764705883 183.53973643060647 L 97.05882352941177 208.85706933582466 L 108.8235294117647 205.79353018827516 L 120.58823529411765 192.79936977395067 L 132.35294117647058 189.55562244125124 L 144.11764705882354 207.34638795392917 L 155.88235294117646 198.04266111847375 L 167.6470588235294 202.69356598202512 L 179.41176470588238 205.04585793073807 L 191.1764705882353 199.80064947786232 L 202.9411764705882 195.51591230966886 L 214.70588235294116 201.1502937581344 L 226.47058823529412 198.22286930362375 L 238.23529411764704 188.02960419253446 L 250.0 199.7680586358671 L 261.7647058823529 189.06292559461775 L 273.52941176470586 191.60117705353866 L 285.2941176470588 188.86546343429387 L 297.05882352941177 185.451093458207